In [36]:
# importing libraries
import pandas as pd
import panel as pn
import hvplot.pandas
import numpy as np
import geopandas as gpd
import datetime

In [66]:
#load_data
df=pd.read_csv("Billionaires Statistics Dataset.csv")
df.sample(3)

,rank,finalWorth,category,personName,age,country,city,source,industries,countryOfCitizenship,...,cpi_change_country,gdp_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,tax_revenue_country_country,total_tax_rate_country,population_country,latitude_country,longitude_country
311,312,7300,Manufacturing,Benu Gopal Bangur,91.0,India,Kolkata,Cement,Manufacturing,India,...,7.7,"$2,611,000,000,000",28.1,113.0,69.4,11.2,49.7,1.366418e+09,20.593684,78.962880
620,611,4500,Manufacturing,Xu Jinfu,59.0,China,Guangzhou,Chemicals,Manufacturing,China,...,2.9,"$19,910,000,000,000",50.6,100.2,77.0,9.4,59.2,1.397715e+09,35.861660,104.195397
1331,1312,2300,Food & Beverage,Jianping Lai,NaN,China,Foshan,Soy sauce,Food & Beverage,China,...,2.9,"$19,910,000,000,000",50.6,100.2,77.0,9.4,59.2,1.397715e+09,35.861660,104.195397


In [67]:
df.columns

Index(['rank', 'finalWorth', 'category', 'personName', 'age', 'country',
       'city', 'source', 'industries', 'countryOfCitizenship', 'organization',
       'selfMade', 'status', 'gender', 'birthDate', 'lastName', 'firstName',
       'title', 'date', 'state', 'residenceStateRegion', 'birthYear',
       'birthMonth', 'birthDay', 'cpi_country', 'cpi_change_country',
       'gdp_country', 'gross_tertiary_education_enrollment',
       'gross_primary_education_enrollment_country', 'life_expectancy_country',
       'tax_revenue_country_country', 'total_tax_rate_country',
       'population_country', 'latitude_country', 'longitude_country'],
      dtype='object')

## Pre Data Processing and cleaning
    1. create a copy of data frame with the useful columns only
    2. check null data
    3. check duplicate


In [68]:
useFulColumns=['rank','finalWorth','personName','age','country','countryOfCitizenship','source',
               'industries','organization','selfMade','gender','birthDate','title',
               'cpi_country','cpi_change_country','gdp_country','gross_tertiary_education_enrollment',
               'gross_primary_education_enrollment_country','life_expectancy_country','total_tax_rate_country'
                  ,'latitude_country','longitude_country']

#create a copy of original data frame
df_copy=df.copy()
df_copy=df_copy[useFulColumns]

In [69]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2640 entries, 0 to 2639
Data columns (total 22 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rank                                        2640 non-null   int64  
 1   finalWorth                                  2640 non-null   int64  
 2   personName                                  2640 non-null   object 
 3   age                                         2575 non-null   float64
 4   country                                     2602 non-null   object 
 5   countryOfCitizenship                        2640 non-null   object 
 6   source                                      2640 non-null   object 
 7   industries                                  2640 non-null   object 
 8   organization                                325 non-null    object 
 9   selfMade                                    2640 non-null   bool   
 10  gender      

In [70]:
# update the age column with the current date
df_copy["birthDate"]=pd.to_datetime(df_copy["birthDate"])
df_copy["age"] = (pd.to_datetime('today')- df_copy['birthDate'])
# convert to year
df_copy["age"]=df_copy["age"].dt.days/365.25

In [81]:
# checking null
df_copy.isnull().sum()

rank                                            0
finalWorth                                      0
personName                                      0
age                                            76
country                                         0
countryOfCitizenship                            0
source                                          0
industries                                      0
organization                                    0
selfMade                                        0
gender                                          0
birthDate                                      76
title                                           0
cpi_country                                   184
cpi_change_country                            184
gdp_country                                   164
gross_tertiary_education_enrollment           182
gross_primary_education_enrollment_country    181
life_expectancy_country                       182
total_tax_rate_country                        182


In [97]:
# null in age
## fill the missing age with the most common age
most_common_age=df_copy["age"].mode().iloc[0]
df_copy["age"].fillna(most_common_age,inplace=True)

## change the data type
df_copy["age"]=df_copy["age"].astype(int)

In [98]:
# null in country

# Identify rows with missing 'country' values
missing_country_rows = df_copy['country'].isnull()

# Fill missing values in 'country' with values from 'country_of_citizenship'
df_copy.loc[missing_country_rows, 'country'] = df_copy.loc[missing_country_rows, 'countryOfCitizenship']

In [99]:
df_copy

,rank,finalWorth,personName,age,country,countryOfCitizenship,source,industries,organization,selfMade,...,title,cpi_country,cpi_change_country,gdp_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,total_tax_rate_country,latitude_country,longitude_country
0,1,211000,Bernard Arnault & family,74,France,France,LVMH,Fashion & Retail,LVMH Moët Hennessy Louis Vuitton,False,...,Chairman and CEO,110.05,1.1,"$2,715,518,274,227",65.6,102.5,82.5,60.7,46.227638,2.213749
1,2,180000,Elon Musk,52,United States,United States,"Tesla, SpaceX",Automotive,Tesla,True,...,CEO,117.24,7.5,"$21,427,700,000,000",88.2,101.8,78.5,36.6,37.090240,-95.712891
2,3,114000,Jeff Bezos,59,United States,United States,Amazon,Technology,Amazon,True,...,Chairman and Founder,117.24,7.5,"$21,427,700,000,000",88.2,101.8,78.5,36.6,37.090240,-95.712891
3,4,107000,Larry Ellison,79,United States,United States,Oracle,Technology,Oracle,True,...,CTO and Founder,117.24,7.5,"$21,427,700,000,000",88.2,101.8,78.5,36.6,37.090240,-95.712891
4,5,106000,Warren Buffett,93,United States,United States,Berkshire Hathaway,Finance & Investments,Berkshire Hathaway Inc. (Cl A),True,...,CEO,117.24,7.5,"$21,427,700,000,000",88.2,101.8,78.5,36.6,37.090240,-95.712891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2635,2540,1000,Yu Rong,51,China,China,Health clinics,Healthcare,Unknown,True,...,Unknown,125.08,2.9,"$19,910,000,000,000",50.6,100.2,77.0,59.2,35.861660,104.195397
2636,2540,1000,"Richard Yuengling, Jr.",80,United States,United States,Beer,Food & Beverage,Unknown,False,...,Unknown,117.24,7.5,"$21,427,700,000,000",88.2,101.8,78.5,36.6,37.090240,-95.712891
2637,2540,1000,Zhang Gongyun,60,China,China,Tyre manufacturing machinery,Manufacturing,Unknown,True,...,Unknown,125.08,2.9,"$19,910,000,000,000",50.6,100.2,77.0,59.2,35.861660,104.195397
2638,2540,1000,Zhang Guiping & family,72,China,China,Real estate,Real Estate,Unknown,True,...,Unknown,125.08,2.9,"$19,910,000,000,000",50.6,100.2,77.0,59.2,35.861660,104.195397


In [100]:
# null in orgainzation and title
organization_null=df_copy["organization"].isnull()
title_null=df_copy["title"].isnull()

# fill the null with Unknown
df_copy["organization"].fillna("Unknown",inplace=True)
df_copy["title"].fillna("Unknown",inplace=True)

In [101]:
# checking duplicates
df_copy.duplicated().any()

False

In [102]:
## saved the processed data
df_copy.to_csv("Processed_data.csv",index=False)